In [9]:
train_file = "/data2/home/zhaoyi/labs/USTC-labs/deeplearn_lab2/dataset/procd/train.csv"
test_file = "/data2/home/zhaoyi/labs/USTC-labs/deeplearn_lab2/dataset/procd/test.csv"
import csv
fp_train = open(train_file,"r")
fp_test = open(test_file,"r")
train_lines = csv.reader(fp_train)
test_lines = csv.reader(fp_test)


In [10]:

train_inputs = list()
train_labels = list()
test_inputs = list()
test_labels = list()

for line in train_lines:
    assert len(line) == 2
    train_inputs.append(line[0])
    train_labels.append(line[1])

for line in test_lines:
    assert len(line) == 2
    test_inputs.append(line[0])
    test_labels.append(line[1])

print(train_inputs[0])
print(train_labels[0])


I wish kids movies were still made this way; dark and deep. There was (get this) character development (and Charlie is the epitome of the dynamic character), plot development, superb animation, emotional involvement, and a rational, relatable, and consistent theme. If not for the handful of song-and-dance routines, you would never have thought this was a kids movie, and this is why I give it such a high rating. This movie is an excellent film, let alone for a kids' movie. Which brings me to my second point: this has got to be the darkest "kids'" movie I've seen in quite some time, this coming from a 22-year-old. I'd be shocked to see any child under the age of 8 not completely terrified throughout a great deal of the latter half and some of the first half of the movie, and it all ends with one of the saddest endings you could ever come across (ala "Jurassic Bark", for those of you who are 'Futurama' fans), and this is what makes this movie so good. Just because the movie universally ev

In [12]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


Loading BERT tokenizer...


In [14]:

MAX_LEN = 128
train_data = tokenizer(train_inputs, padding = "max_length", max_length = MAX_LEN, truncation=True ,return_tensors = "pt")

In [15]:
test_data = tokenizer(test_inputs, padding = "max_length", max_length = MAX_LEN, truncation=True ,return_tensors = "pt")

In [16]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [25]:
print(train_data['attention_mask'][1000])
for i in range(len(train_labels)):
    train_labels[i] = int(train_labels[i])
for i in range(len(test_labels)):
    test_labels[i] = int(test_labels[i])

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])


In [26]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 16
train = TensorDataset(train_data["input_ids"], train_data["attention_mask"], torch.tensor(train_labels))
train_sampler = RandomSampler(train)
train_dataloader = DataLoader(train, sampler=train_sampler, batch_size=batch_size)

test = TensorDataset(test_data["input_ids"], test_data["attention_mask"], torch.tensor(test_labels))
test_sampler = RandomSampler(test)
test_dataloader = DataLoader(test, sampler=test_sampler, batch_size=batch_size)

from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 1
from transformers import get_scheduler
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


In [27]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
for epoch in range(num_epochs):
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 10 == 0 and not step == 0:
            print("step: ",step, "  loss:",total_loss/(step*batch_size))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()        
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)

        loss = outputs.loss       
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        lr_scheduler.step()
    avg_train_loss = total_loss / len(train_dataloader)      
    print("avg_loss:",avg_train_loss)

step:  10   loss: 0.04339771568775177
step:  20   loss: 0.04191254135221243
step:  30   loss: 0.038629327652355036
step:  40   loss: 0.038163122814148666
step:  50   loss: 0.03524111438542604
step:  60   loss: 0.034875927555064364
step:  70   loss: 0.03374068486903395
step:  80   loss: 0.03332722702762112
step:  90   loss: 0.032741141795284215
step:  100   loss: 0.032245917301625016
step:  110   loss: 0.031243460189381785
step:  120   loss: 0.03091991146793589
step:  130   loss: 0.03001235188701405
step:  140   loss: 0.029513178170392555
step:  150   loss: 0.029327894986296694
step:  160   loss: 0.029058256311691365
step:  170   loss: 0.028571795877617073
step:  180   loss: 0.02817363610568767
step:  190   loss: 0.027926233358142014
step:  200   loss: 0.02761917853844352
step:  210   loss: 0.027439205730999155
step:  220   loss: 0.0278678647168404
step:  230   loss: 0.02774512844828322
step:  240   loss: 0.02749694231218503
step:  250   loss: 0.027694021173752845
step:  260   loss: 0.0

In [35]:
import numpy as np
model.eval()
tot_corrs = 0
tot_num = 0
with torch.no_grad():  

    for step, batch in enumerate(train_dataloader):
        bat_input_ids = batch[0].to(device)
        bat_att_masks = batch[1].to(device)
        batch_labels = batch[2].tolist()
        outputs = model(bat_input_ids, 
                    token_type_ids=None, 
                    attention_mask=bat_att_masks)
        preds = torch.argmax(outputs["logits"],dim=1).tolist()
        assert len(preds) == len(batch_labels)
        corrs = sum([batch_labels[i]==preds[i] for i in range(len(preds))])
        tot_corrs += corrs
        tot_num += len(preds)
print('--testing phase---: testing acc = ', tot_corrs/tot_num)  
        
        


--testing phase---: testing acc =  0.9365721279199702
